In [8]:
import pyarrow.parquet as pq
import pandas as pd
from IPython.display import display

#dataset_1 = pq.ParquetDataset('dataset-1/w32processes')
#dataset_2 = pq.ParquetDataset('dataset-2/w32processes')
#dataset_3 = pq.ParquetDataset('dataset-3/w32processes')
dataset_4 = pq.ParquetDataset('dataset-1/w32processes')

#table_1 = dataset_1.read()
#table_2 = dataset_2.read()
#table_3 = dataset_3.read()
table_4 = dataset_4.read()

#df_1 = table_1.to_pandas()
#df_2 = table_2.to_pandas()
#df_3 = table_3.to_pandas()
df = table_4.to_pandas()

#df_1.to_csv('test_1.csv')
#df_2.to_csv('test_2.csv')
#df_3.to_csv('test_3.csv')
#df_4.to_csv('test_4.csv')

pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [9]:
#display(df)

In [10]:
n_elevated_processes = df[df['username'].str.contains('NT AUTHORITY')]
print('Nº of elevated processes: ' + str(len(n_elevated_processes)))

Nº of elevated processes: 1181


In [11]:
# Elevated powershell / cmd processes:
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('cmd')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 2


,hostname,name,arguments,username,path,pid
179,416QOMNW,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\System32\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,200
30,476QRISB,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\SysWOW64\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,4652


In [12]:
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('powershell')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 0


,hostname,name,arguments,username,path,pid


In [13]:
# Count on how many different hostnames appears each process
host_count = n_elevated_processes.groupby('name')['hostname'].nunique()

# Rare processes, less than 5 hosts
rare_procs = host_count[host_count < 5].index.tolist()
rare_elevated = n_elevated_processes[n_elevated_processes['name'].isin(rare_procs)]

display(rare_elevated)

print('There are no unique processes appearing in less than 5 hostnames')

,hostname,name,arguments,username,path,pid
179,416QOMNW,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\System32\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,200
30,476QRISB,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\SysWOW64\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,4652
81,501UZWNC,procdump.exe,C:\Windows\SysWOW64\procdump.exe -ma lsass.exe lsass_dump,NT AUTHORITY\SYSTEM,C:\Windows\SysWOW64\,1356
97,538OJQEN,procdump.exe,C:\Program Files (x86)\Common Files\procdump.exe -ma lsass.exe lsass_dump,NT AUTHORITY\SYSTEM,C:\Program Files (x86)\Common Files\,5556
4,540TIMSW,taskhostex.exe,taskhostex.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32,636
61,540TIMSW,wmiprvse.exe,C:\Windows\system32\wbem\wmiprvse.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32\wbem,5368
42,771SQOEB,procdump.exe,C:\Program Files\Common Files\Internet Explorer\procdump.exe -ma lsass.exe lsass_dump,NT AUTHORITY\SYSTEM,C:\Program Files\Common Files\Internet Explorer\,2856
15,994LJYNN,taskhostex.exe,taskhostex.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32,2060
20,994LJYNN,wmiprvse.exe,C:\Windows\system32\wbem\wmiprvse.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32\wbem,1908


There are no unique processes appearing in less than 5 hostnames


In [18]:
# windows services
known_services = [
    'services.exe', 'svchost.exe', 'lsass.exe', 'wininit.exe',
    'taskhostw.exe', 'spoolsv.exe', 'smss.exe', 'csrss.exe',
    'dllhost.exe', 'dcomlaunch.exe', 'winlogon.exe', 'wmiprvse.exe']

service_like = df[
    (df['name'].str.lower().isin(known_services)) #|
    #df['path'].str.lower().str.contains(r'\\system32\\', na=False) |
    #(df['path'].str.lower().str.contains(r'\\windows\\.*service', na=False)) |
    #(df['path'].str.lower().str.contains(r'\\windows\\.*svchost', na=False))
]

service_like = service_like[
    service_like['arguments'].notna() & 
    (service_like['arguments'].str.strip() != '')
]

# Solo servicios elevados
service_like_elevated = service_like[service_like['username'].str.contains('NT AUTHORITY')]

# Agrupar y contar en cuántos hostnames aparece cada proceso
service_host_counts = service_like_elevated.groupby('name')['hostname'].nunique()

# Procesos raros
rare_services = service_host_counts[service_host_counts < 5].index.tolist()

# Filtrar servicios sospechosos
susp_services = service_like_elevated[service_like_elevated['name'].isin(rare_services)]

display(service_like_elevated)
print(service_host_counts)

,hostname,name,arguments,username,path,pid
4,111RWHSW,svchost.exe,c:\windows\system32\svchost.exe -k localservice -p -s LicenseManager,NT AUTHORITY\SYSTEM,c:\windows\system32,2808
5,111RWHSW,svchost.exe,c:\windows\system32\svchost.exe -k dcomlaunch -p -s LSM,NT AUTHORITY\SYSTEM,c:\windows\system32,1960
6,111RWHSW,svchost.exe,c:\windows\system32\svchost.exe -k dcomlaunch -p -s PlugPlay,NT AUTHORITY\SYSTEM,c:\windows\system32,196
7,111RWHSW,svchost.exe,c:\windows\system32\svchost.exe -k appmodel -p -s StateRepository,NT AUTHORITY\SYSTEM,c:\windows\system32,4704
8,111RWHSW,svchost.exe,c:\windows\system32\svchost.exe -k localservice -p -s FontCache,NT AUTHORITY\SYSTEM,c:\windows\system32,3100
...,...,...,...,...,...,...
35,994LJYNN,svchost.exe,C:\Windows\system32\svchost.exe -k LocalService,NT AUTHORITY\SYSTEM,C:\Windows\system32,1996
38,994LJYNN,svchost.exe,C:\Windows\system32\svchost.exe -k netsvcs,NT AUTHORITY\SYSTEM,C:\Windows\system32,6136
47,994LJYNN,svchost.exe,C:\Windows\system32\svchost.exe -k DcomLaunch,NT AUTHORITY\SYSTEM,C:\Windows\system32,656
49,994LJYNN,svchost.exe,C:\Windows\system32\svchost.exe -k LocalServiceNoNetwork,NT AUTHORITY\SYSTEM,C:\Windows\system32,2948


name
DllHost.exe       6
WinLogon.exe      6
csrss.exe        12
dllhost.exe      10
lsass.exe        25
services.exe     12
smss.exe         12
spoolsv.exe      25
svchost.exe      25
taskhostw.exe    11
wininit.exe      14
winlogon.exe     19
wmiprvse.exe      2
Name: hostname, dtype: int64


In [22]:
# Añadir procesos cuyo nombre parezca de servicio
suspicious_keywords = ['svc', 'service', 'host', 'win', 'sys']
service_like_extra = df[
    df['name'].str.lower().str.contains('|'.join(suspicious_keywords))
]

# Solo los que son elevados
service_like_extra_elevated = service_like_extra[
    service_like_extra['username'].str.contains('NT AUTHORITY')
]

# Procesos raros en menos de 5 máquinas
host_counts_extra = service_like_extra_elevated.groupby('name')['hostname'].nunique()
rare_extra = host_counts_extra[host_counts_extra < 5].index.tolist()

# Resultado extendido
susp_services_extended = service_like_extra_elevated[
    service_like_extra_elevated['name'].isin(rare_extra)
]

# Combinar ambas detecciones (precisas y heurísticas)
all_suspicious_services = pd.concat([susp_services, susp_services_extended]).drop_duplicates()
display(all_suspicious_services)

,hostname,name,arguments,username,path,pid
61,540TIMSW,wmiprvse.exe,C:\Windows\system32\wbem\wmiprvse.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32\wbem,5368
20,994LJYNN,wmiprvse.exe,C:\Windows\system32\wbem\wmiprvse.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32\wbem,1908
4,540TIMSW,taskhostex.exe,taskhostex.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32,636
15,994LJYNN,taskhostex.exe,taskhostex.exe,NT AUTHORITY\SYSTEM,C:\Windows\system32,2060
